In [91]:
using RDatasets, LinearAlgebra

Let's compare how Poisson EPCA performs against vanilla PCA on compositional data. For this example, we use the [Chemical Composition of Pottery](https://vincentarelbundock.github.io/Rdatasets/doc/carData/Pottery.html) dataset from RDatasets. More information about the dataset can be found on the doc. 

In [92]:
soils = dataset("car", "Pottery")
labels = Vector(soils[:, 1])
compositions = Matrix(soils[:, 2:end])
normalized_compositions = compositions ./ sum(compositions, dims=2)

26×5 Matrix{Float64}:
 0.546282  0.265554   0.163126   0.00569044   0.0193475
 0.560976  0.287805   0.139431   0.00487805   0.00691057
 0.563707  0.273745   0.149807   0.00501931   0.00772201
 0.48299   0.267535   0.236875   0.00671987   0.00587988
 0.518797  0.265414   0.200752   0.0075188    0.0075188
 0.518554  0.297812   0.165081   0.00808754   0.0104662
 0.531579  0.224211   0.224211   0.0105263    0.00947368
 0.490901  0.244604   0.250106   0.00761744   0.00677105
 0.511521  0.252995   0.208295   0.0133641    0.0138249
 0.478059  0.246878   0.257938   0.0099893    0.00713521
 ⋮                                            
 0.837308  0.126656   0.0333863  0.000529942  0.00211977
 0.889328  0.0741107  0.0331028  0.000494071  0.00296443
 0.873362  0.0912179  0.0329937  0.000485201  0.00194081
 0.896552  0.0650862  0.0310345  0.00301724   0.00431034
 0.907692  0.0574359  0.0287179  0.00307692   0.00307692
 0.905045  0.0563798  0.0331355  0.00296736   0.0024728
 0.917079  0.0505217  0.

In [93]:
using Revise
using ExpFamilyPCA

In [94]:
epca = PoissonEPCA()
A1 = fit!(epca, normalized_compositions; maxoutdim=3, verbose=true, maxiter=20)
Xre1 = decompress(epca, A)

Iteration: 1/20 | Loss: 1.1906795777415402
Iteration: 10/20 | Loss: 0.040218941006744516
Iteration: 20/20 | Loss: 0.034536886863912195


26×5 Matrix{Float64}:
 0.556976  0.267691   0.171382   0.00844586  0.00928941
 0.5643    0.285017   0.140637   0.00595775  0.00715463
 0.556028  0.276763   0.146026   0.00615782  0.00722424
 0.486302  0.270059   0.230344   0.00963931  0.00959326
 0.514841  0.268243   0.198478   0.00871272  0.00910731
 0.541308  0.300338   0.172134   0.00754753  0.0086219
 0.50868   0.219271   0.215575   0.0103699   0.00982097
 0.48038   0.244849   0.242217   0.0104813   0.00990697
 0.510686  0.250528   0.212267   0.00982573  0.0098037
 0.475889  0.247087   0.256547   0.0113141   0.0105228
 ⋮                                           
 0.839833  0.12534    0.0339065  0.00177103  0.00258914
 0.878189  0.0747451  0.0303142  0.00190268  0.00241077
 0.862827  0.0941695  0.0294158  0.0015961   0.00221385
 0.902971  0.0649694  0.0330551  0.00253879  0.00297904
 0.909828  0.0555374  0.02946    0.00218594  0.00252212
 0.904918  0.0567702  0.032003   0.00249673  0.00282414
 0.93351   0.0518958  0.0296744  0.0024

In [95]:
using MultivariateStats

In [96]:
M = fit(PCA, normalized_compositions; maxoutdim=3)
A2 = predict(M, normalized_compositions)
Xre2 = reconstruct(M, A2)

26×5 Matrix{Float64}:
 0.547151  0.248102   0.184304    0.0100833     0.0103599
 0.562799  0.251017   0.184079    0.000908299   0.00119714
 0.565004  0.24758    0.181563    0.00278131    0.00307266
 0.482157  0.284357   0.216459    0.00840603    0.00862165
 0.518758  0.266193   0.199805    0.00749671    0.00774617
 0.520019  0.268247   0.200962    0.00526089    0.00551098
 0.530165  0.25274    0.189585    0.013624      0.0138855
 0.489295  0.277021   0.210763    0.0113491     0.0115721
 0.511095  0.261589   0.197865    0.0146034     0.0148475
 0.476329  0.28178    0.21558     0.01305       0.0132613
 ⋮                                             
 0.838196  0.108728   0.0551447  -0.00130695   -0.000762323
 0.889138  0.0779635  0.0284262   0.00193981    0.00253266
 0.873525  0.0879417  0.0369696   0.000492827   0.00107078
 0.896235  0.0714741  0.0232815   0.00420446    0.00480446
 0.907266  0.0660332  0.0182839   0.00390331    0.00451351
 0.904492  0.067519   0.0196165   0.00388231    0

We can immediately see that Poisson EPCA does better at compressing composition data. The PCA reconstruction contains negative values. Surprisingly, the reconstructed rows sums to around one in both.

In [97]:
sum(Xre1, dims=2)'

1×26 adjoint(::Matrix{Float64}) with eltype Float64:
 1.01378  1.00307  0.992199  1.00594  0.999382  …  1.02028  0.995643  0.99458

In [98]:
sum(Xre2, dims=2)'

1×26 adjoint(::Matrix{Float64}) with eltype Float64:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [99]:
using Statistics

In [100]:
kl(p, q) = sum(p .* log.(p ./ q))

# TODO: perform this analysis w/ different numbers of bases
kl1 = []
kl2 = []
for (p1, p2, q) in zip(eachrow(Xre1), eachrow(Xre2), eachrow(normalized_compositions))
    if any(p2 .< 0)
        continue
    end
    push!(kl1, kl(p1, q))
    push!(kl2, kl(p2, q))
end
@show mean(kl1)
@show mean(kl2);

mean(kl1) = 0.0017301866573429777
mean(kl2) = 0.004455817802528197
